<div align=left style="width: 200; height: 80px; overflow: hidden">
    <img src=http://static1.squarespace.com/static/571446ff60b5e92c3a2b4249/57d8a40b9de4bb459f731cf3/58cb2f229de4bb4a049d38c2/1505340359463/teselaGenlogo.jpg align=right width=200>
</div>

# Hello World! DESIGN Module

This notebook shows how to use the TeselaGen's Python API Client to interact with the DESIGN module.

We start by making some imports

In [1]:
from pathlib import Path
import platform

from IPython.display import display
from IPython.core.display import HTML
import nglview
import pandas as pd

from teselagen.api import TeselaGenClient
from teselagen.utils.plot_tools import plot_plasmid_features
from teselagen.utils.plot_tools import RenderJSON

print(f"python version     : {platform.python_version()}")
print(f"pandas version     : {pd.__version__}")

python version     : 3.11.6
pandas version     : 2.2.3


And then login into the platform. You should get "*Connection Accepted*" printed below. 

In [2]:
# Connect to your teselagen instance by passing it as the 'host_url' argument of TeselaGenClient(host_url=host_url)
# client = TeselaGenClient(host_url="https://your-instance-name.teselagen.com")
client = TeselaGenClient()
client.login()
client.select_laboratory(lab_name="The Test Lab")

Client ready. Please login
Connection Accepted at http://host.docker.internal:3000
Selected Lab: The Test Lab


In [3]:
print(client.host_url)

http://host.docker.internal:3000


## Downloading DNA sequences

In this section we are going to download and explore a sample sequence from the DESIGN module. This sequences is named `GFP_UV`. In the next cell we are going to download this sequence. You can use the cell's output to explore the contents of this object:

In [4]:
sequence = client.design.get_dna_sequences(name='GFP_UV')[0]
RenderJSON(sequence)

The output contained  a list of all sequences named `'GFP_UV'`. We just got the first one and now we check the   features it contains

In [5]:
features = sequence['features']
for feat in features:
    print(feat['name'])

araC
operator O2
araC promoter
operator O1
CAP site
Operator I2 and I1
pBAD promoter
RBS
GFPuv
ccmN_sig_pep
gly_ser_linker
XhoI_silent_mutation
BamHI_silent_mutation
ssrA tag enhanced
ssrA tag
dbl term
pSC101**
T0
CmR


In [6]:
features

[{'notes': {},
  'id': 'c1f69f91-a91a-41e9-be30-9fece45a0073',
  'start': 0,
  'end': 0,
  'type': 'CDS',
  'name': 'araC',
  'strand': -1,
  '__typename': 'sequenceFeature',
  'annotationTypePlural': 'features',
  'forward': False,
  'color': '#EF6500'},
 {'notes': {},
  'id': 'e746be40-4c3d-45ec-9c6f-c17d199ca67d',
  'start': 0,
  'end': 0,
  'type': 'protein_bind',
  'name': 'operator O2',
  'strand': 1,
  '__typename': 'sequenceFeature',
  'annotationTypePlural': 'features',
  'forward': True,
  'color': '#2E2E2E'},
 {'notes': {},
  'id': 'd0197504-4052-45f3-a01c-e27c8711cca3',
  'start': 0,
  'end': 0,
  'type': 'promoter',
  'name': 'araC promoter',
  'strand': -1,
  '__typename': 'sequenceFeature',
  'annotationTypePlural': 'features',
  'forward': False,
  'color': '#31B440'},
 {'notes': {},
  'id': 'ab056d21-3841-49e1-8914-1fad6d077df3',
  'start': 0,
  'end': 0,
  'type': 'protein_bind',
  'name': 'operator O1',
  'strand': 1,
  '__typename': 'sequenceFeature',
  'annotationT

Each element contains all the information about that particular feature. In the following cell we show the contents of the `GFPuv` feature (only if available):

In [7]:
gfp_uv_feature = [feat for feat in features if feat['name'] == "GFPuv"]
if gfp_uv_feature:
    display(gfp_uv_feature[0])

{'notes': {},
 'id': 'd81d6164-7597-41bc-b913-205f558837e1',
 'start': 0,
 'end': 0,
 'type': 'CDS',
 'name': 'GFPuv',
 'strand': 1,
 '__typename': 'sequenceFeature',
 'annotationTypePlural': 'features',
 'forward': True,
 'color': '#EF6500'}

We can use the above object to get the precise nucleotide sequence for that feature:

In [8]:
if gfp_uv_feature:
    display(sequence['sequence'][int(gfp_uv_feature[0]['start']):int(gfp_uv_feature[0]['end']) + 1])

''

We can also make a plot of all features by using [dna_features_viewer library](https://github.com/Edinburgh-Genome-Foundry/DnaFeaturesViewer) (see plot_plasmid_features implementation for details). As there are many features we will just focus on the biggest ones (> 100 base pairs)

In [9]:
features=[feat for feat in features if feat['end'] - feat['start'] > 100]
features

[]

In [10]:
filtered_features = [feat for feat in features if feat['end'] - feat['start'] > 100]
if filtered_features:
    _ = plot_plasmid_features(plasmid_length=len(sequence['sequence']),
                            features=filtered_features)
    # This line just centers the image
    HTML("""<style> .output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>""")
else:
    print("No features found")

No features found


## Downloading designs

Now we'll download a design from the platform

In [11]:
# We first obtain designs id by its name
design_name = "CGG Design demo notebook"
design_info = client.design.get_designs(name=design_name)
if not design_info:
    print("Design not found, please change design_name with an existent design name")
else:
    print("Downloaded design", design_info)

Design not found, please change design_name with an existent design name


Click the following link to see the design in the platform

In [12]:
if design_info:
    design_url = f"{client.host_url}/design/client/designs/{design_info['id']}"
    display(HTML(f"""<a href="{design_url}" target="_blank" rel="noopener noreferrer">{design_url}</a>"""))

In [13]:
# Then download design:
design = None
if design_info:
    design = client.design.get_design(design_info['id'])


You can use the output of the next cell to explore the design object

In [14]:
if design:
    RenderJSON(design)

For design upload, please refer to the *Closing-the-DBTL-Cycle.ipynb* jupyter notebook.

## Working with amino acid sequences

Below we will explore how to upload and download amino acid secuences with an Antimicrobial Peptide (AMP). 
 

The [2KNJ](https://www.rcsb.org/structure/2KNJ) peptide has AMP properties. Its sequence is: ```HHQELCTKGDDALVTELECIRLRISPETNAAFDNAVQQLNCLNRACAYRKMCATNNLEQAMSVYFTNEQIKEIHDAATACDPEAHHEHDH```

The next cell shows its 3D structure:

(Note: For displaying the molecule, the [nglview](https://github.com/nglviewer/nglview) library is being used. If the figure is not showing, try running the following command from your environment terminal and reload: `jupyter-nbextension enable nglview --py --sys-prefix`. If still doesn't work you may want to explore their [FAQ](https://github.com/nglviewer/nglview/blob/master/docs/FAQ.md#widget-not-shown))

In [17]:
# Uncomment the following lines to show the 3D structure
# view = nglview.show_pdbid("2KNJ")  # load "2KNJ" from RCSB PDB and display viewer widget
# view

In the next cell we are going to upload this amino acid sequence into the DESIGN module.

If upload works ok, the endpoint will return something like:

```
{'createdAminoAcidSequences': [{'id': '17', 'name': '2KNJ2'}]}
```

However, the current example is already loaded in the default server so, in this case, the endpoint will return a non-empty `existingAminoAcidSequences` field, as occurs when some of the uploaded sequences matches a sequence from the the dataset. 

In [18]:
result = client.design.import_aa_sequences([{
    'AA_NAME': "2KNJ2",
    'AA_SEQUENCE': 'HHQELCTKGDDALVTELECIRLRISPETNAAFDNAVQQLNCLNRACAYRKMCATNNLEQAMSVYFTNEQIKEIHDAATACDPEAHHEHDH'
}])
result

{'createdAminoAcidSequences': [{'id': '266ad981-3b0e-4d49-8fcf-823b860173ff',
   'name': '2KNJ2'}]}

In [19]:
result

{'createdAminoAcidSequences': [{'id': '266ad981-3b0e-4d49-8fcf-823b860173ff',
   'name': '2KNJ2'}]}

Now we'll download the sequence. Amino acid sequence `id` is needed for this:

In [20]:
# The following list contains a list with the uploaded sequence id
if "existingAminoAcidSequences" in result:
    ids_list = [result['existingAminoAcidSequences'][0]['id']]
elif "createdAminoAcidSequences" in result:
    ids_list = [result['createdAminoAcidSequences'][0]['id']]
else:
    raise AssertionError(f"Received malformed result: {result}")
result = client.design.export_aa_sequences(ids_list)

## Finally, the downloaded object can be seen below

In [21]:
RenderJSON(result)